In [1]:
import numpy as np

In [2]:
from jitr import xs, reactions, rmatrix

In [3]:
from elm import chuq

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [4]:
from jitr.optical_potentials import chuq as jchuq

In [5]:
Ca48 = (48,20)
proton  =(1,1)

In [6]:
params = jchuq.get_samples_democratic(proton)

In [7]:
chuq_mean = np.mean(params, axis=0)

In [8]:
reaction = reactions.ElasticReaction(
    Ca48,
    proton,
)
Elab = 35

In [9]:
ws = xs.elastic.DifferentialWorkspace.build_from_system(
    reaction=reaction,
    kinematics=reaction.kinematics(Elab),
    channel_radius_fm=20,
    solver=rmatrix.Solver(40),
    lmax=30,
    angles=np.linspace(0.01,np.pi,100)
)

In [10]:
coul, centr, so = jchuq.calculate_params(projectile=proton, target=Ca48, Elab=Elab)
xs_jchuq = ws.xs(
    interaction_central=jchuq.central_plus_coulomb,
    interaction_spin_orbit=jchuq.spin_orbit,
    args_central=(centr, coul),
    args_spin_orbit=so
)

In [11]:
centr0, centr1, so0, so1, coul, alpha = chuq.calculate_params(projectile=proton, target=Ca48, Elab=Elab)
xs_chuq = ws.xs(
    interaction_central=chuq.central_plus_coulomb,
    interaction_spin_orbit=chuq.spin_orbit,
    args_central=(alpha, coul, centr0, centr1),
    args_spin_orbit=(alpha, so0, so1)
)

In [14]:
np.testing.assert_allclose(xs_chuq.dsdo, xs_jchuq.dsdo)
np.testing.assert_allclose(xs_chuq.Ay, xs_jchuq.Ay)